## 1. wordset 제작

In [36]:
import pandas as pd
from konlpy.tag import Okt
import re
from collections import Counter
from tqdm import tqdm
import pandas as pd
import ast

In [54]:
num2target = ['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']
num2target_dict = {idx:item for idx, item in enumerate(num2target)}
target2num = {item:idx for idx, item in enumerate(num2target)}

# 2018 빅카인즈로 말뭉치 세트만들기(키워드 기반)
def save_wordset(df, target):
    print(f"{target} is now processing...")
    df_target = df[df['target2num']==target]
    keywords = []
    for _, item in tqdm(df_target.iterrows(), desc=f"{target}에 대한 카운트중입니다"):
        keywords.extend(item['keyword'].split(','))

    keywords_dict = {item:count for item, count in Counter(keywords).items()}
    keyword_counted = sorted(Counter(keywords_dict).items(), key=lambda x:-x[1])
        
    keyword = [item[0] for item in keyword_counted]
    count = [item[1] for item in keyword_counted]
    index = [i for i in range(len(count))]
    
    world_dataframe = pd.DataFrame({'id':index, 'keyword':keyword, 'count':count})
    world_dataframe.to_csv("../data/bigkinds_2018/wordset_"+target+"_2018.csv", index=False)
    print(f"{target} is done\n")
    

# train으로 주어진 데이터로 말뭉치 세트 만들기(okt.nouns 기반)
okt = Okt()

def save_wordset_klue(df, target):
    print(f"{target}, {num2target[target]} is now processing...")
    df_target = df[df['target']==target]
    nouns = []
    for _, item in tqdm(df_target.iterrows(), desc=f"{target}에 대한 카운트중입니다"):
        _nouns = okt.nouns(item['text'])
        # _nouns = [item for item in _nouns if len(item)>1]
        nouns.extend(_nouns)

    keywords_dict = {item:count for item, count in Counter(nouns).items()}
    keyword_counted = sorted(Counter(keywords_dict).items(), key=lambda x:-x[1])
        
    keyword = [item[0] for item in keyword_counted]
    count = [item[1] for item in keyword_counted]
    index = [i for i in range(len(count))]
    
    world_dataframe = pd.DataFrame({'id':index, 'keyword':keyword, 'count':count})
    world_dataframe.to_csv("../data/bigkinds_2018/wordset_"+str(num2target[target])+"_klue.csv", index=False)
    print(f"{target} is done\n")
    

def make_wordset():
    wordset = []
    for target in num2target:
        df = pd.read_csv(f"../data/bigkinds_2018/wordset_{target}_2018.csv")
        _wordset = df['keyword'].tolist()
        _count = df['count'].tolist()
        # _count = [i for i in range(len(_wordset))]
                
        wordset.append(dict(zip(_wordset, _count)))
    return wordset


def make_wordset_klue():
    wordset = []
    for target in num2target:
        df = pd.read_csv(f"../data/bigkinds_2018/wordset_{target}_klue_copy.csv")
        _wordset = df['keyword'].tolist()
        _count = df['count'].tolist()
        # _count = [i for i in range(len(_wordset))]
                
        wordset.append(dict(zip(_wordset, _count)))
    return wordset


def wordset_count(wordsets, noun_counted, threshold=10, verbose=False):
    count = [0]*len(wordsets)
    
    for idx, wordset in enumerate(wordsets):
        for item, _ in noun_counted.items():
            if item in list(wordset.keys()) and wordset[item] >= threshold:
                count[idx] += 1 #-1e-5*wordset[item]
                if verbose:
                    print(f"item {item} is counted in {num2target[idx]}")
                
    return count

# 전체 train data에 대해 적용해보자.
# 제목 count가 같은경우 본문으로 count 해보기
def label_compare(df, wordset, threshold=100):
    max_count_label_text, max_count_label_content = [0]*len(df), [0]*len(df)
    
    for idx, item in tqdm(df.iterrows(), desc="max word 카운팅 중..."):
        text, target, content = item['text'], item['target'], item['crawling_content']
        t_counted_nouns = Counter(okt.nouns(text))
        c_counted_nouns = Counter(okt.nouns(content))
        
        t_count = wordset_count(wordset, t_counted_nouns, threshold=10, verbose=False)
        c_count = wordset_count(wordset, c_counted_nouns, threshold=threshold, verbose=False)
        
        t_result = list(filter(lambda x:t_count[x]==max(t_count), range(len(t_count))))
        c_result = list(filter(lambda x:c_count[x]==max(c_count), range(len(c_count))))

        # max_count_label_text.append(t_result)
        # max_count_label_content.append(c_result)
        max_count_label_text[idx] = (t_result)
        max_count_label_content[idx] = (c_result)
        
    df['target_from_text'] = max_count_label_text
    df['target_from_content'] = max_count_label_content
    
    return df


# voting 결과 똑같거나, 모두 다르면 그냥 그 텍스트 제거
def target_voting(df):
    target_nunique = df['target'].nunique()
    columns = ['target', 'pre_model_target', 'wordset_target']
    voted_target = []
    
    for _, item in tqdm(df.iterrows(), desc='voting 중 입니다...'):
        vote = [0]*target_nunique
        
        for col in columns:
            _labels = item[col]
            if type(_labels) == list:
                for i in _labels:
                    vote[i] += 1
            elif type(_labels) == str:
                _labels = ast.literal_eval(_labels)
                for i in _labels:
                    vote[i] += 1
            else: 
                vote[_labels] += 1
                
        if not vote.count(max(vote)) > 1:
            result = vote.index(max(vote))
        else:
            result = "ERROR"

        voted_target.append(result)
    
    df['voted_target'] = voted_target
    removed = df[df['voted_target'] == 'ERROR'].reset_index().copy()
    after = df[df['voted_target'] != 'ERROR'].reset_index().copy()
    
    return after, removed

In [ ]:
## (2018 빅카인즈) wordset to csv

# train_data = pd.read_csv("../data/train_2018_concat_v1.csv").iloc[45678:]
# for target in train_data['target2num'].unique().tolist():
#     make_wordset(train_data, target)

In [ ]:
# train으로 주어진 데이터로 말뭉치 세트 만들기(okt.nouns 기반)

# setting
num2target = ['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']
num2target_dict = {idx:item for idx, item in enumerate(num2target)}
target2num = {item:idx for idx, item in enumerate(num2target)}

# # data
# train_data = pd.read_csv("../data/train_spelling_v2.csv")
# for target in train_data['target'].unique():
#     save_wordset_klue(train_data, target)

In [ ]:
# okt 기반으로, 의심되는 df(label_v1)의 target을 새롭게 설정해보기

# data loading
train_data = pd.read_csv("../data/train_crawling_content.csv")
train_data_label_v1 = pd.read_csv("../data/train_spelling_v2_label_v1.csv")

train_target_not_same = train_data[train_data['target']!=train_data_label_v1['target']].reset_index().copy()
train_target_not_same['pre_model_target'] = train_data_label_v1.loc[train_data['target']!=train_data_label_v1['target'], :]['target'].tolist()

# # function call
# wordset = make_wordset_klue()

# train_target_not_same_nouns = label_compare(train_target_not_same, wordset, threshold=150)
# train_target_not_same_nouns.to_csv("../data/bigkinds_2018/train_data_nouns.csv", index=False)

In [55]:
# voting

train_target_not_same_nouns_v2 = pd.read_csv("../data/bigkinds_2018/train_data_nouns_v2.csv")
train_target_not_same_nouns_after, removed = target_voting(train_target_not_same_nouns_v2)

voting 중 입니다...: 2612it [00:00, 6984.34it/s]


In [56]:
print(len(train_target_not_same_nouns_after), len(removed), len(train_target_not_same_nouns_v2))

1885 727 2612


In [72]:
num2target

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [59]:
train_data.loc[train_data['ID'].isin(train_target_not_same_nouns_after['ID']), :]


,ID,text,target,url,date
42,ynat-v1_train_00042,제임스 떠난 NBA 클리블랜드 1승 10패…전체 꼴찌,4,https://sports.news.naver.com/news.nhn?oid=001...,2018.11.08 15:06
85,ynat-v1_train_00085,메시 통산 4번째 유럽 골든슈 최종 확정…호날두와 동률,4,https://sports.news.naver.com/news.nhn?oid=001...,2017.05.29 08:09
98,ynat-v1_train_00098,LG전자 휴대전화 국내 생산 중단…베트남으로 거점 이동종합,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.04.24. 오후 4:32
109,ynat-v1_train_00109,서울에 다시 오존주의보…도심·서북·동북권 발령종합,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.07.27. 오후 3:41
144,ynat-v1_train_00144,아프리카돼지열병 파동에 中 돼지고기 수입량 76% 급증,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.09.24. 오후 4:26
...,...,...,...,...,...
45482,ynat-v1_train_45482,해외서도 내 데이터 그대로…SKT T괌·사이판패스 출시종합,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.09.12. 오전 11:11
45535,ynat-v1_train_45535,수원 아시아 4강 이끈 특급 선방 신화용 분석·조언의 힘,3,https://sports.news.naver.com/news.nhn?oid=001...,2018.09.19 22:38
45560,ynat-v1_train_45560,오늘 고강도 부동산 대책 발표…다주택자 등 투기수요 겨냥,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.08.02. 오전 5:01
45600,ynat-v1_train_45600,伊 대통령 새 연정 승인…콘테 현 총리에 연정 구성 권한 부여,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.08.29. 오후 6:21


In [68]:
train_data.loc[train_data['ID'].isin(train_target_not_same_nouns_after['ID']), 'target'] = train_target_not_same_nouns_after['voted_target'].tolist()
train_data.loc[train_data['ID'].isin(train_target_not_same_nouns_after['ID']), :]

,ID,text,target,url,date
42,ynat-v1_train_00042,제임스 떠난 NBA 클리블랜드 1승 10패…전체 꼴찌,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.11.08 15:06
85,ynat-v1_train_00085,메시 통산 4번째 유럽 골든슈 최종 확정…호날두와 동률,5,https://sports.news.naver.com/news.nhn?oid=001...,2017.05.29 08:09
98,ynat-v1_train_00098,LG전자 휴대전화 국내 생산 중단…베트남으로 거점 이동종합,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.04.24. 오후 4:32
109,ynat-v1_train_00109,서울에 다시 오존주의보…도심·서북·동북권 발령종합,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.07.27. 오후 3:41
144,ynat-v1_train_00144,아프리카돼지열병 파동에 中 돼지고기 수입량 76% 급증,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.09.24. 오후 4:26
...,...,...,...,...,...
45482,ynat-v1_train_45482,해외서도 내 데이터 그대로…SKT T괌·사이판패스 출시종합,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.09.12. 오전 11:11
45535,ynat-v1_train_45535,수원 아시아 4강 이끈 특급 선방 신화용 분석·조언의 힘,5,https://sports.news.naver.com/news.nhn?oid=001...,2018.09.19 22:38
45560,ynat-v1_train_45560,오늘 고강도 부동산 대책 발표…다주택자 등 투기수요 겨냥,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.08.02. 오전 5:01
45600,ynat-v1_train_45600,伊 대통령 새 연정 승인…콘테 현 총리에 연정 구성 권한 부여,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.08.29. 오후 6:21


In [71]:
train_data.to_csv("../data/train_spelling_v2_wordset_v1.csv", index=False)

In [69]:
train_org = pd.read_csv("../data/train_spelling_v2.csv")
len(train_data[train_org['target']!=train_data['target']])

1530

In [63]:
train_target_not_same_nouns_after['voted_target']

0       5
1       5
2       0
3       3
4       4
       ..
1880    0
1881    5
1882    6
1883    6
1884    0
Name: voted_target, Length: 1885, dtype: object

In [57]:
train_data.loc[train_data['ID'] == train_target_not_same_nouns_after['ID'], :])

ValueError: Can only compare identically-labeled Series objects

In [45]:
# 저장
# removed -> 변경하지 않는다.
# train_target_not_same_nouns_after -> voted target으로 변경한다.

train_data = pd.read_csv("../data/train_spelling_v2.csv")
# train_data = train_data[train_data['ID'] == train_target_not_same_nouns_after['ID']]

In [35]:
# vote - text, body 기반 하나의 컬럼으로 정렬
import ast

def vote_text_body(df):
    wordset_target = []
    columns = ['target_from_text', 'target_from_content']
    
    for index, item in tqdm(df.iterrows(), desc='text, content voting 중 입니다...'):
        target_nunique = df['target'].nunique()
        vote = [0]*target_nunique
        
        for col in columns:
            _labels = item[col]
            if type(_labels) == list:
                for i in _labels:
                    vote[i] += 1
            elif type(_labels) == str:
                _labels = ast.literal_eval(_labels)
                for i in _labels:
                    vote[i] += 1
            else: 
                vote[_labels] += 1

        result = list(filter(lambda x:vote[x]==max(vote), range(len(vote))))
        wordset_target.append(result)
        
    return wordset_target


train_target_not_same_nouns_v2 = pd.read_csv("../data/bigkinds_2018/train_data_nouns_v2.csv")
train_target_not_same_nouns_v2['wordset_target'] = vote_text_body(train_target_not_same_nouns_v2)
train_target_not_same_nouns_v2.to_csv("../data/bigkinds_2018/train_data_nouns_v2.csv", index=False)

text, content voting 중 입니다...: 2612it [00:00, 5197.23it/s]


In [33]:
num2target

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [34]:
train_target_not_same_nouns_v2.head(5)

,index,ID,text,target,url,date,crawling_content,pre_model_target,target_from_text,target_from_content,wordset_target
0,16,ynat-v1_train_00016,박원순 시장 아부다비 루브르 박물관 방문,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.05.02. 오전 9:30,(서울=연합뉴스) 아랍에미리트(UAE)를 방문 중인 박원순 서울시장이 1일(현지시간...,2,"[3, 6]","[2, 3]",[3]
1,42,ynat-v1_train_00042,제임스 떠난 NBA 클리블랜드 1승 10패…전체 꼴찌,4,https://sports.news.naver.com/news.nhn?oid=001...,2018.11.08 15:06,오클라호마시티 선더 알렉스 아브리네스(가운데)가 8일(한국시각) 미국 오하이오주 ...,5,[5],[5],[5]
2,49,ynat-v1_train_00049,반민특위 임무 아직 끝나지 않았다,5,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.03. 오후 4:11,한길사 '반민특위의 역사적 의미를 다시 묻는다' 출간\n\n\n\n (서울=연...,2,"[0, 1, 2, 3, 4, 5, 6]",[6],[6]
3,85,ynat-v1_train_00085,메시 통산 4번째 유럽 골든슈 최종 확정…호날두와 동률,4,https://sports.news.naver.com/news.nhn?oid=001...,2017.05.29 08:09,바르셀로나의 골잡이 리오넬 메시.[EPA=연합뉴스 자료 사진](서울=연합뉴스) 이영...,5,"[0, 5, 6]",[5],[5]
4,95,ynat-v1_train_00095,창원 39사단 유니시티 1순위 청약에 20만명 몰려,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.01. 오후 12:55,39사단 터 전경 \n (창원=연합뉴스) 이정훈 기자 = 경남 창원시 의창구 ...,2,"[1, 5]",[2],"[1, 2, 5]"


In [9]:
# test
train_target_not_same_nouns = pd.read_csv("../data/bigkinds_2018/train_data_nouns.csv")
train_target_not_same_nouns.head(5)

# 1. target이 틀리고 pre_model_target 이 맞을 경우
# 2. target이 맞을 경우

,index,ID,text,target,url,date,crawling_content,pre_model_target,target_from_text,target_from_content
0,16,ynat-v1_train_00016,박원순 시장 아부다비 루브르 박물관 방문,6,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.05.02. 오전 9:30,(서울=연합뉴스) 아랍에미리트(UAE)를 방문 중인 박원순 서울시장이 1일(현지시간...,2,"[3, 6]","[0, 3, 6]"
1,42,ynat-v1_train_00042,제임스 떠난 NBA 클리블랜드 1승 10패…전체 꼴찌,4,https://sports.news.naver.com/news.nhn?oid=001...,2018.11.08 15:06,오클라호마시티 선더 알렉스 아브리네스(가운데)가 8일(한국시각) 미국 오하이오주 ...,5,[5],[5]
2,49,ynat-v1_train_00049,반민특위 임무 아직 끝나지 않았다,5,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.06.03. 오후 4:11,한길사 '반민특위의 역사적 의미를 다시 묻는다' 출간\n\n\n\n (서울=연...,2,"[0, 1, 2, 3, 4, 5, 6]",[6]
3,85,ynat-v1_train_00085,메시 통산 4번째 유럽 골든슈 최종 확정…호날두와 동률,4,https://sports.news.naver.com/news.nhn?oid=001...,2017.05.29 08:09,바르셀로나의 골잡이 리오넬 메시.[EPA=연합뉴스 자료 사진](서울=연합뉴스) 이영...,5,[5],[5]
4,95,ynat-v1_train_00095,창원 39사단 유니시티 1순위 청약에 20만명 몰려,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.05.01. 오후 12:55,39사단 터 전경 \n (창원=연합뉴스) 이정훈 기자 = 경남 창원시 의창구 ...,2,[5],[4]
